In [ ]:
# Power & Energy from CSV (+ optional Siglent comparison)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

%matplotlib inline
print("pandas", pd.__version__)

# --- Helpers ---
def clean_columns(df):
    out = df.copy()
    out.columns = (
        out.columns
          .str.strip()
          .str.replace(r"\s+\(.*?\)", "", regex=True)   # drop units in ()
          .str.replace(r"[^0-9a-zA-Z_]+", "_", regex=True)  # spaces → _
          .str.lower()
    )
    return out

def find_col_like(df, keywords):
    for c in df.columns:
        if any(k in c.lower() for k in keywords):
            return c
    return None

def coerce_units(series, name):
    name = name.lower()
    s = pd.to_numeric(series, errors="coerce")
    if "mv" in name: s = s/1000
    if "ma" in name: s = s/1000
    return s

def integrate_energy_wh(ts, p_w):
    t = pd.to_datetime(ts)
    p = pd.to_numeric(p_w, errors="coerce")
    d = pd.DataFrame({"t": t, "p": p}).dropna().sort_values("t")
    if d.empty: return pd.Series([], dtype=float), 0.0
    dt = d["t"].diff().dt.total_seconds().fillna(0)
    e_wh = ((d["p"].shift(1)+d["p"])/2) * dt / 3600.0
    return e_wh.cumsum().fillna(0), e_wh.cumsum().iloc[-1]

In [ ]:
pi_csv = "energy_log_20250829_120521_521_cpu1.csv"  # adjust filename

df_raw = pd.read_csv(pi_csv)
df = clean_columns(df_raw)
print("Columns:", df.columns.tolist())

time_col = find_col_like(df, ["time","timestamp","date"])
volt_col = find_col_like(df, ["volt","v"])
curr_col = find_col_like(df, ["curr","amp","a","i"])
print("Detected:", time_col, volt_col, curr_col)

V = coerce_units(df[volt_col], volt_col)
I = coerce_units(df[curr_col], curr_col)
P = V * I

d = pd.DataFrame({
    "time": pd.to_datetime(df[time_col]),
    "voltage_v": V,
    "current_a": I,
    "power_w": P
}).dropna().sort_values("time").set_index("time")

cum_Wh, total_Wh = integrate_energy_wh(d.index, d["power_w"])
d["energy_Wh"] = cum_Wh
print("Total energy (Wh):", round(total_Wh, 6))

In [ ]:
plt.figure()
d["power_w"].plot()
plt.title("Pi: Power over time (W)")
plt.ylabel("W"); plt.xlabel("time"); plt.tight_layout(); plt.show()

plt.figure()
d["energy_Wh"].plot()
plt.title("Pi: Cumulative energy (Wh)")
plt.ylabel("Wh"); plt.xlabel("time"); plt.tight_layout(); plt.show()

In [ ]:
siglent_csv = "siglent_export.csv"

s_raw = pd.read_csv(siglent_csv)
s = clean_columns(s_raw)
print("Siglent cols:", s.columns.tolist())

st = find_col_like(s, ["time","timestamp","date"])
sp = find_col_like(s, ["power","w"])
sig = pd.DataFrame({"time": pd.to_datetime(s[st]),
                    "siglent_power_w": pd.to_numeric(s[sp], errors="coerce")}).dropna()
sig = sig.sort_values("time").set_index("time")

# resample both to 1s for fair comparison
pi1 = d[["power_w"]].resample("1S").mean()
sg1 = sig.resample("1S").mean()
comp = pi1.join(sg1, how="inner").dropna()

print(comp.head())
print("Correlation:", comp["power_w"].corr(comp["siglent_power_w"]))

plt.figure()
comp["power_w"].plot(label="Pi")
comp["siglent_power_w"].plot(label="Siglent")
plt.legend(); plt.title("Power comparison (Pi vs Siglent)")
plt.ylabel("W"); plt.tight_layout(); plt.show()